<a href="https://colab.research.google.com/github/sumitdas1984/aisome-2023-cognitive-coders/blob/main/DeBERTA_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install pandas
!pip install scikit-learn
!pip install sentencepiece
!pip install protobuf==3.20.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 2.11.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you have protobuf 3.20.0 which is incompatible.


In [2]:
pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.8 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import transformers
import torch
device = torch.device("cuda")

In [5]:
device

device(type='cuda')

In [6]:
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback

In [7]:
train = pd.read_csv("/content/drive/MyDrive/train_df.csv")
test = pd.read_csv('/content/drive/MyDrive/val_df.csv')


In [8]:
train.head()

,Unnamed: 0.1,Unnamed: 0,ID,tweet,labels,label_list,label_len,tweet_len,clean_tweets,tokens,neg,neu,pos,compound
0,1477,1477,1352994869556350977t,#TRUSTCCP? Do you trust the Chinese Communist ...,country,['country'],1,32,trustccp do you trust the chinese communist pa...,"['trustccp', 'trust', 'chinese', 'communist', ...",0.035,0.679,0.286,0.8738
1,1790,1790,1364574998397132805t,@hannah62687 @mgtalbott @DrTomFrieden For many...,ineffective unnecessary,"['ineffective', 'unnecessary']",2,50,hannah62687 mgtalbott drtomfrieden for many ge...,"['hannah62687', 'mgtalbott', 'drtomfrieden', '...",0.139,0.756,0.105,-0.2295
2,2615,2615,1334085299945807873t,Imagine if the covid vaccine has a side effect...,side-effect,['side-effect'],1,17,imagine if the covid vaccine has a side effect...,"['imagine', 'covid', 'vaccine', 'side', 'effec...",0.000,1.000,0.000,0.0000
3,6969,6969,1321645107183980544t,"@GreyHaired07 and yet, every other year we are...",mandatory,['mandatory'],1,31,greyhaired07 and yet every other year we are r...,"['greyhaired07', 'yet', 'every', 'year', 'remi...",0.151,0.849,0.000,-0.5859
4,7172,7172,1432922183337066498t,#VAERS 17y♂️#Pfizer #Covidvaccine #Suicide att...,side-effect,['side-effect'],1,45,vaers 17ypfizer covidvaccine suicide attempt p...,"['vaers', '17ypfizer', 'covidvaccine', 'suicid...",0.092,0.857,0.051,-0.4588


In [9]:
import ast
y_train = train['label_list'].apply(ast.literal_eval)
y_test = test['label_list'].apply(ast.literal_eval)

In [10]:
from sklearn.preprocessing import MultiLabelBinarizer
# Binarize the labels for multi-label classification
mlb = MultiLabelBinarizer()
# y_dataset_binarized = mlb.fit_transform(y_dataset)

y_train_binarized = mlb.fit_transform(y_train)
y_test_binarized = mlb.fit_transform(y_test)

In [11]:
train['one_hot_labels'] = list(y_train_binarized)
test['one_hot_labels'] = list(y_test_binarized)
train.head(2)

,Unnamed: 0.1,Unnamed: 0,ID,tweet,labels,label_list,label_len,tweet_len,clean_tweets,tokens,neg,neu,pos,compound,one_hot_labels
0,1477,1477,1352994869556350977t,#TRUSTCCP? Do you trust the Chinese Communist ...,country,['country'],1,32,trustccp do you trust the chinese communist pa...,"['trustccp', 'trust', 'chinese', 'communist', ...",0.035,0.679,0.286,0.8738,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,1790,1790,1364574998397132805t,@hannah62687 @mgtalbott @DrTomFrieden For many...,ineffective unnecessary,"['ineffective', 'unnecessary']",2,50,hannah62687 mgtalbott drtomfrieden for many ge...,"['hannah62687', 'mgtalbott', 'drtomfrieden', '...",0.139,0.756,0.105,-0.2295,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1]"


In [12]:
train_df, test_df = train, test
print(train_df.shape)
print(test_df.shape)


(7936, 15)
(1985, 15)


In [13]:
from transformers import DebertaV2Config, DebertaV2Model, DebertaTokenizer
MODEL_PATH = 'microsoft/deberta-large'
MAX_LENGTH = 128

In [14]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

In [15]:
train_encodings = tokenizer(train_df["tweet"].values.tolist(), max_length=MAX_LENGTH, padding="max_length",truncation=True,return_tensors='pt')
test_encodings = tokenizer(test_df["tweet"].values.tolist(), max_length=MAX_LENGTH, padding="max_length",truncation=True,return_tensors='pt')

In [16]:

train_labels = train_df["one_hot_labels"].values.tolist()
test_labels = test_df["one_hot_labels"].values.tolist()

In [17]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # labels need to be converted to float for multilabel loss calculation
        item['labels'] = torch.tensor(self.labels[idx]).float()
        return item

    def __len__(self):
        return len(self.labels)

In [18]:
train_dataset = CustomDataset(train_encodings, train_labels)
test_dataset = CustomDataset(test_encodings, test_labels)

In [19]:
train_dataset[0]

<ipython-input-17-731660984850>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'input_ids': tensor([    1, 10431,  6997, 11120,  3376,   510,   116,  1832,    47,  2416,
             5,  1111, 12416,  1643,     7,   146, 21688, 47412,   128, 39150,
           833, 35661,  2647,     6,   205,     6,  1303,    89,    32,  2213,
             9,    49, 12987, 34426,  1952,   447,   543,    13,    47,   235,
           122,    11,  1776, 17914,   734,  1205,   640,    90,     4,   876,
            73,   398,   510,   771,   401,   605,   791,   288,   725, 16291,
             2,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [20]:

# sanity check
tokenizer.decode(train_dataset[0]["input_ids"])


<ipython-input-17-731660984850>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


"[CLS]#TRUSTCCP? Do you trust the Chinese Communist Party to make YOUR mRNA 'vaccine'? Well, good, cause there are hundreds of their virologists working hard for you right now in Big Pharma... https://t.co/8PW6wU0Hkl[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]"

In [21]:
# id2label = { 0: "unnecessary", 1: "mandatory", 2: "pharma", 3 : "conspiracy", 4:"political", 5:"country", 6:"rushed", 7:"ingredients", 8:"side-effect", 9:"ineffective", 10:"religious", 11:"none"}
# label2id = { "unnecessary": 0, "mandatory": 1, "pharma": 2, "conspiracy": 3,  "political": 4, "country": 5, "rushed": 6, "ingredients": 7, "side-effect":8, "ineffective": 9, "religious": 10, "none": 11}
label_cols=list(mlb.classes_)
label2id = dict()
id2label = dict()
for idx, col in enumerate(label_cols):
    id2label[idx] = col
    label2id[col] = idx

In [22]:
from scipy.special import expit
from sklearn import metrics
from sklearn.metrics import classification_report

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds = np.array(expit(preds) > thresh, dtype=int)
    macro_f1 = metrics.f1_score(labels, preds, average='macro', zero_division=0)
    micro_f1 = metrics.f1_score(labels, preds, average='micro', zero_division=0)
    weighted_f1 = metrics.f1_score(labels, preds, average='weighted', zero_division=0)
    samples_f1 = metrics.f1_score(labels, preds, average='samples', zero_division=0)
    acc_thresh = accuracy_thresh(preds, labels, thresh)
    return {"macro_f1":macro_f1, "micro_f1":micro_f1,
            "weighted_f1":weighted_f1, "samples_f1":samples_f1,
            'accuracy_thresh': acc_thresh}

def accuracy_thresh(y_pred, y_true, thresh, sigmoid=True):
    y_pred = torch.from_numpy(y_pred)
    y_true = torch.from_numpy(y_true)
    if sigmoid:
        y_pred = y_pred.sigmoid()
    return ((y_pred>thresh)==y_true.bool()).float().mean().item()

def get_binary_preds(y_probs, thresh):
    if y_probs > thresh:
        return 1
    else:
        return 0
get_binary_preds_vec = np.vectorize(get_binary_preds)

def generate_performance_metrics(test_dataset,label_classes):
    predictions = trainer.predict(test_dataset)
    all_preds = predictions.predictions
    all_preds = torch.from_numpy(all_preds).float()
    preds = all_preds.sigmoid()
    test_bin_preds = get_binary_preds_vec(preds.numpy(), thresh)

    tdf_1hot = pd.DataFrame(predictions.label_ids, columns=label_classes)
    labels_subset_ids = [label2id[c] for c in label_classes if tdf_1hot[c].sum()>0]
    labels_subset = [c for c in label_classes if tdf_1hot[c].sum()>0]

    all_metrics = classification_report(predictions.label_ids, test_bin_preds, digits=3, labels = labels_subset_ids,
                                       target_names=labels_subset, output_dict=True, zero_division=0)
    metrics_df = pd.DataFrame(all_metrics).transpose().reset_index()
    return metrics_df

In [23]:
num_labels = len(id2label)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH, num_labels=num_labels, label2id=label2id, id2label=id2label, problem_type="multi_label_classification").to(device)

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-large and are newly initialized: ['pooler.dense.weight', 'classifier.bias', 'classifier.weight', 'pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
batch_size = 10
# configure logging so we see training loss
logging_steps = len(train_dataset) // batch_size
args = TrainingArguments(
    output_dir="model-deberta",
    evaluation_strategy = "epoch",
    save_strategy = 'epoch',
    learning_rate=2e-5,
    fp16=True,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=15,
    #weight_decay=0.01,
    logging_steps=logging_steps,
    gradient_accumulation_steps= 32,
    lr_scheduler_type='cosine',
    metric_for_best_model='macro_f1',
    load_best_model_at_end = True
)

In [25]:

trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5, early_stopping_threshold=0.001)]
)

In [26]:

thresh=0.2
trainer.train()

<ipython-input-17-731660984850>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
You're using a DebertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Macro F1,Micro F1,Weighted F1,Samples F1,Accuracy Thresh
0,No log,0.291435,0.046320,0.343834,0.172696,0.350210,0.103233
1,No log,0.288081,0.046320,0.343602,0.172696,0.350210,0.103233
2,No log,0.255978,0.166326,0.426448,0.372091,0.460497,0.103233
3,No log,0.211682,0.319643,0.576258,0.529116,0.588111,0.103233
4,No log,0.175676,0.466185,0.660793,0.649169,0.679664,0.103233
5,No log,0.161913,0.513564,0.691009,0.681495,0.716908,0.103233
6,No log,0.152269,0.547468,0.712034,0.702149,0.732028,0.103233
7,No log,0.149358,0.591823,0.728894,0.722513,0.743807,0.103233
8,No log,0.150563,0.601776,0.724024,0.722049,0.746164,0.103233
9,No log,0.147730,0.620558,0.741603,0.738274,0.756761,0.103233


<ipython-input-17-731660984850>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-17-731660984850>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-17-731660984850>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-17-731660984850>:7: UserWarning: To copy construct from a 

RuntimeError: ignored

In [ ]:

test_result = trainer.evaluate()
test_result